In [1]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QuantumVolume

from src.GCP_hypergraph import QuantumCircuitHyperGraph
from circuits.cp_fraction import cp_fraction
from circuits.QAOA import QAOA_random
from src.FM_main import *
from src.multilevel_FM import *

def build_circuit(num_qubits,fraction,group_gates=True):
    try:
        circuit = cp_fraction(num_qubits,num_qubits,fraction)
        # circuit = QFT(num_qubits,do_swaps=False)
        # circuit = QAOA_random(num_qubits,0.5,reps=1)
        # circuit = QuantumVolume(num_qubits,num_qubits)
            
        circuit = transpile(circuit, basis_gates = ['u','cp'])
        graph = QuantumCircuitHyperGraph(num_qubits,depth=circuit.depth())
        graph.map_circuit_to_hypergraph(circuit,group_gates=group_gates)

        return graph
    except IndexError:
        print("Conversion issue, retrying")
        return build_circuit(num_qubits,fraction,group_gates=True)
    except TypeError:
        print("Conversion issue, retrying")
        return build_circuit(num_qubits,fraction,group_gates=True)

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FM_QV_5it.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FM_QV_5it.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 49, 8)

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 2 + i

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        circuit = QuantumVolume(num_qubits,num_qubits)
        circuit = transpile(circuit, basis_gates = ['u','cp'])
        base_graph = QuantumCircuitHyperGraph(num_qubits,depth=circuit.depth())
        base_graph.map_circuit_to_hypergraph(circuit,group_gates=True)
        depth = base_graph.depth
        initial_assignment = set_initial_partitions(qpu_info,num_qubits, depth ,num_partitions, reduced=True)
        
        # -------------------------
        # 2. Fine-grained partitioning
        # -------------------------
        graph_list = [base_graph]
        mapping_list = [{i : set([i]) for i in range(depth)}]

        assignment_list_f, cost_list_f, time_list_f = multilevel_FM(graph_list,
                                                                mapping_list,
                                                                initial_assignment,
                                                                qpu_info,
                                                                limit = num_qubits*depth/8,
                                                                pass_list= [50],
                                                                stochastic=True,
                                                                lock_nodes=False,
                                                                log = False,
                                                                add_initial = False,
                                                                costs = None)
        total_time_f = sum(time_list_f)
        min_cost_f = min(cost_list_f)
        # -------------------------
        # 3. Window-based refinement
        # -------------------------

        assignment_list_w, cost_list_w, time_list_w = MLFM_window(base_graph, 
            num_levels=8,
            initial_assignment=initial_assignment,  
            qpu_info= qpu_info, 
            limit = None, 
            pass_list= None, 
            stochastic=True, 
            lock_nodes=True,
            log = False,
            add_initial = False,
            costs = None)
        
        total_time_w = sum(time_list_w)
        min_cost_w = min(cost_list_w)

        
        # -------------------------
        # 4. Block refinement
        # -------------------------
        assignment_list_b, cost_list_b, time_list_b = MLFM_blocks(base_graph,
                                        num_blocks=8,
                                        initial_assignment=initial_assignment,  
                                        qpu_info= qpu_info, 
                                        limit = None, 
                                        pass_list= None, 
                                        stochastic=True, 
                                        lock_nodes=True,
                                        log = False,
                                        add_initial = False,
                                        costs = None,
                                        full = False)
        
        total_time_b = sum(time_list_b)
        min_cost_b = min(cost_list_b)
        
        # -------------------------
        # 5. Recursive refinement
        # -------------------------
        assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                    initial_assignment,  
                                    qpu_info, 
                                    limit = None, 
                                    pass_list= None, 
                                    stochastic=True, 
                                    lock_nodes=False,
                                    log = False,
                                    add_initial = False,
                                    costs = None)
        
        total_time_r = sum(time_list_r)
        min_cost_r = min(cost_list_r)

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "f_cost": min_cost_f,
            "w_cost": min_cost_w,
            "b_cost": min_cost_b,
            "r_cost":  min_cost_r,
            "time_f": total_time_f,
            "time_w": total_time_w,
            "time_b": total_time_b,
            "time_r": total_time_r,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------
    f_cost_list = [x["f_cost"] for x in iteration_data]
    w_cost_list = [x["w_cost"] for x in iteration_data]
    b_cost_list = [x["b_cost"] for x in iteration_data]
    r_cost_list = [x["r_cost"] for x in iteration_data]
    
    f_time_list = [x["time_f"] for x in iteration_data]
    w_time_list = [x["time_w"] for x in iteration_data]
    b_time_list = [x["time_b"] for x in iteration_data]
    r_time_list = [x["time_r"] for x in iteration_data]
    
    mean_f_cost = float(np.mean(f_cost_list))
    mean_w_cost = float(np.mean(w_cost_list))
    mean_b_cost = float(np.mean(b_cost_list))
    mean_r_cost = float(np.mean(r_cost_list))
    
    mean_f_time = float(np.mean(f_time_list))
    mean_w_time = float(np.mean(w_time_list))
    mean_b_time = float(np.mean(b_time_list))
    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 10 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")
    print(f"  `Fine`:    {mean_f_cost:.3f}")
    print(f"  Window: {mean_w_cost:.3f}")
    print(f"  Block: {mean_b_cost:.3f}")
    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")
    print(f"  Fine:    {mean_f_time:.3f}")
    print(f"  Window: {mean_w_time:.3f}")
    print(f"  Block: {mean_b_time:.3f}")
    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,
        "mean_f_cost": mean_f_cost,
        "mean_w_cost": mean_w_cost,
        "mean_b_cost": mean_b_cost,
        "mean_r_cost": mean_r_cost,
        "mean_f_time": mean_f_time,
        "mean_w_time": mean_w_time,
        "mean_b_time": mean_b_time,
        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

SyntaxError: invalid syntax (2938908441.py, line 10)